In [ ]:
import pandas as pd
import re
from ast import literal_eval
from collections import Counter

In [2]:
notes = pd.read_csv('../data/notes-00000.tsv', sep='\t')
ratings = pd.read_csv('../data/ratings-00000.tsv', sep='\t')

In [ ]:
merge_corpus = pd.read_csv("../data/merged3_mturk.csv")

In [ ]:
merge_corpus.columns

In [4]:
len(set(merge_corpus.tweetId))

2208

In [5]:
print(pd.crosstab(merge_corpus['credibility'],merge_corpus['classification']))

classification   MISINFORMED_OR_POTENTIALLY_MISLEADING  NOT_MISLEADING
credibility                                                           
credible                                           209              25
mostly_credible                                     56              14
not_credible                                      1983             184
not_verifiable                                     300              25
uncertain                                          225              22


In [6]:
pd.crosstab(merge_corpus['credibility'],merge_corpus['classification']).sum().sum()

3043

In [8]:
merge_corpus['joined_text']=merge_corpus['CR Fact'].apply(lambda x: " ".join(literal_eval(x)))

In [9]:
def find_winner(x):
    x = Counter(x).items()
    m = max([xx[1] for xx in x ])
    winner = [xx[0] for xx in x if xx[1]==m]
    if len(winner)==2: return 'Tie'
    else: return winner[0]
    #return ",".join(sorted([xx[0] for xx in x if xx[1]==m]))

In [10]:
d=merge_corpus.groupby(['Tweet','tweetId','credibility','joined_text']).agg({'classification':[find_winner]})

In [11]:
d=d.reset_index()

In [13]:
pd.crosstab(d['credibility'],d['classification']['find_winner'])

find_winner,MISINFORMED_OR_POTENTIALLY_MISLEADING,NOT_MISLEADING,Tie
credibility,,,
credible,126,9,9
mostly_credible,44,7,5
not_credible,1476,62,55
not_verifiable,225,8,9
uncertain,156,8,9


In [14]:
tie_df = d[d['classification']['find_winner']=='NOT_MISLEADING']

In [16]:
for i,x in merge_corpus[(merge_corpus.credibility=='credible') & (merge_corpus.classification=='MISINFORMED_OR_POTENTIALLY_MISLEADING') ].iterrows():
    print(f'Tweet: {x.Tweet}')
    print()
    print(f'CR Fact: {" ".join(literal_eval(x["CR Fact"]))}')
    print()
    print(f'Note: {x.summary}')
    print()
    print()
    print()
    print()
    

Tweet: Chicago PD Says Enhanced Vid Shows Gun in 13-Year-Old Adam Toledo's Hand https://t.co/B0Twu733RL

CR Fact: Chicago Mayor Lori Lightfoot said Adam Toledo had a gun in his hand when he was fatally shot by a police officer, or words to that effect.

Note: Adam Toledo did not have the gun in his hand when he was approached by the police who shot him. He has his arms up and complied. The gun was on the floor, not his hands.




Tweet: Hey look, the Roger Chamberlain--the chair of the Senate Education Committee who has retweeted Nazis and promoted white nationalist content--is flashing the white power symbol on the Minnesota Senate floor. #mnleg https://t.co/54IMLTMLTS

CR Fact: A photograph shows Roger Stone and members of the Proud Boys flashing a white power symbol.

Note: Making an &quot;OK&quot; or &quot;0&quot; sign with your hand is not a symbol of white supremacy. That rumor began as a trolling attempt on 4chan, called &quot;Operation O-KKK&quot; to trick the public and news o

In [17]:
for i,x in merge_corpus[(merge_corpus.credibility=='not_credible') & (merge_corpus.classification=='MISINFORMED_OR_POTENTIALLY_MISLEADING') ].iterrows():
    print(f'Tweet: {x.Tweet}')
    print()
    print(f'CR Fact: {" ".join(literal_eval(x["CR Fact"]))}')
    print()
    print(f'Note: {x.summary}')
    print()
    print()
    print()
    print()
    

Tweet: Trump won the election by a landslide

CR Fact: Donald Trump won the 2020 election, by a lot

Note: According to numerous independent sources, Trump lost the election.    Politifact, 1/6/21: https://www.politifact.com/factchecks/2021/jan/07/donald-trump/trump-clings-fantasy-landslide-victory-egging-supp/    &quot;All 50 states and the District of Columbia have certified their election results, which Congress sought to finalize Jan. 6? There is no evidence that voter fraud affected that outcome.&quot;




Tweet: DONALD TRUMP WON THE 2020 ELECTION BY A LOT!

#DonaldTrump https://t.co/Wgk60OUZg1

CR Fact: Donald Trump won the 2020 election, by a lot

Note: According to numerous independent sources, Trump lost the election.    Politifact, 1/6/21: https://www.politifact.com/factchecks/2021/jan/07/donald-trump/trump-clings-fantasy-landslide-victory-egging-supp/    &quot;All 50 states and the District of Columbia have certified their election results, which Congress sought to finalize 

Tweet: Without evidence, MSNBC's Mike Barnicle blamed former President Donald Trump for the Atlanta-area massage parlor shootings. https://t.co/7wUEgJw3ww

CR Fact: [ ' T h e   N e w   Y o r k   T i m e s   s t a t e d ,   a s   f a c t ,   t h a t   H i l l a r y   C l i n t o n   a n d   G e o r g e   S o r o s   h a d   b e e n   r e s p o n s i b l e   f o r   p a y i n g   a   w o m a n   t o   m a k e   f a l s e   a l l e g a t i o n s   o f   s e x u a l   a s s a u l t   a g a i n s t   D o n a l d   T r u m p . ' ]

Note: The article from Breitbart that is attached to this Tweet was accurate and informed.  However, the actual tweet is misleading about the content of its own article.  




Tweet: My thoughts as the last U.S. military personnel left #Afghanistan todayat the direction of the Presidentleaving hundreds of Americans who wanted to be rescued trapped behind Taliban-controlled lines.  https://t.co/x7sfYslnAg

CR Fact: [ ' " 1 2 2   v i c i o u s   p r i s o n e r s , 


Note: The Bill of Rights is written to be all inclusive. The 4th Amendment applies to your locked phone, even if the word &quot;phone&quot; is not in the text.




Tweet: Just to remind everyone, the individual right to own and use firearms is completely made up and the product of NRA funded activism.

CR Fact: [ ' C l a i m   t h a t   G e o r g e   W a s h i n g t o n   s a i d ,   " W h e n   a   n a t i o n   m i s t r u s t s   i t s   c i t i z e n s   w i t h   g u n s   i t   i s   i t   s e n d i n g   a   c l e a r   m e s s a g e .   I t   n o   l o n g e r   t r u s t s   i t s   c i t i z e n s   b e c a u s e   s u c h   a   g o v e r n m e n t   h a s   e v i l   p l a n s . " ' ]

Note: The constitution predates the NRA. He?s just a liar.     https://www.law.cornell.edu/wex/second_amendment    




Tweet: Just to remind everyone, the individual right to own and use firearms is completely made up and the product of NRA funded activism.

CR Fact: [ ' C l a i m   t h a t 




Tweet: President Biden told us he would have a plan to defeat the virus on day 1. He just never told us that it would be to give the vaccine to terrorists before most Americans.

CR Fact: [ ' J o e   B i d e n   s a i d   i f   h e   i s   e l e c t e d   p r e s i d e n t ,   h e   w i l l   l o c k   d o w n   t h e   c o u n t r y   u n t i l   a   v a c c i n e   i s   r e a d y   a n d   e v e r y o n e   i s   v a c c i n a t e d ' ]

Note: Per the pentagon, no prisoners at Guantanamo Bay have been vaccinated. https://twitter.com/PentagonPresSec/status/1355600353086025731?s=20




Tweet: Bidens tax plan will raise taxes on 75% of middle-class families next year and that number increases to 95% by 2031.

No thank you! 

The American people can spend their hard-earned money better than a bloated federal government.

CR Fact: [ ' D u r i n g   h i s   2 0 2 0   p r e s i d e n t i a l   c a m p a i g n ,   D e m o c r a t i c   n o m i n e e   J o e   B i d e n   p r o p o s e d 



Tweet: Louis DeJoy wrecked the US Postal Service to try to rig the election for Donald Trump. He failed. But the mail is still broken, and he's still in charge of it. Fire DeJoy. https://t.co/T2hKkZcijS

CR Fact: [ ' A   U S P S   C a r r i e r   W a s   R e c e n t l y   C a u g h t   S t e a l i n g   A   T r u m p - P e n c e   C a m p a i g n   E l e c t i o n   S i g n ' ]

Note: The article provides absolutely no evidence or facts regarding its claim. The article in question is a blatant lie.




Tweet: Ivermectin works

CR Fact: [ ' T h e   m e d i c i n e   s h o w n   i n   t h e   v i r a l   p o s t   c a n   c u r e   s t o n e s   i n   4   h o u r s ' ]

Note: Ivermectin is not currently approved for use to treat COVID-19 in the United States. This guidance may change, but there is not enough evidence at this time of its effectiveness or subsequent side effects.    1. https://www.fda.gov/consumers/consumer-updates/why-you-should-not-use-ivermectin-treat-or-prevent-covid





Tweet: Trump did a cringeworthy ad for the Moonies, a religious cult

He put Amy Coney Barrett, a Handmaid of the religious cult People of Praise, on the Supreme Court

Evangelicals endorsed him 

Guy doesnt give a damn about religion but is bankrolled &amp; supported by religious lunatics

CR Fact: [ " T r u m p ' s   p h o t o   o p   i n   f r o n t   o f   a   c h u r c h   h o l d i n g   a   B i b l e   w a s   m i m i c k i n g   A d o l f   H i t l e r   d o i n g   t h e   s a m e " ]

Note: Amy Coney Barrett is not a handmaid. The people of praise is not a religious cult. Donald Trump did not put her on the Supreme Court; rather, she was appointed by Trump and confirmed by the Senate. Lastly, there is no evidence that Trump is bankrolled by religious lunatics.    https://www.google.com/amp/s/www.wsj.com/amp/articles/what-is-people-of-praise-amy-coney-barretts-faith-brings-attention-to-a-charismatic-group-11602445444




Tweet: US: Vaccine deaths reported by year. 

Nothin


CR Fact: [ ' T h e   C D C   S i m p l y   M a d e   3 0 , 0 0 0   C O V I D - 1 9   D e a t h s   D i s a p p e a r   I n   S c a m   O n   A m e r i c a n s ' ]

Note: This has already been debunked. If Twitter wants to defend factual info, not misinformation, they should reject this.    https://www.nationalreview.com/news/yahoo-story-on-florida-covid-study-misrepresented-key-finding-studys-author-says/




Tweet: Good news: Mild COVID-19 induces lasting antibody protection https://t.co/79nTY4XxKm via @wustlmed

CR Fact: [ ' V i r a l   C O V I D - 1 9   t e s t s   g r a p h   s h o w i n g   U S   r e a c h i n g   h e r d   i m m u n i t y ' ]

Note: This appears to be misrepresenting the stated science. It appears to say, ?just get casual COVID and you?ll be immunized?, where it?s actually saying, ?even if you only had a mild case, you?re likely immune.    It?s not suggesting exposing yourself to the virus to gain immunity. https://medicine.wustl.edu/news/good-news-mild-covid-19